In [6]:
import pandas as pd
import os
import json

In [7]:
def combine_json_to_dataframe(folder_path):
    all_dataframes = []  # List to store individual DataFrames

    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)

            # Load JSON data
            with open(file_path, 'r') as file:
                data = json.load(file)

            # Convert JSON data to DataFrame
            if 'auctions' in data:
                df = pd.json_normalize(data, 'auctions')

                # Add the file_date as a new column
                df['datetime'] = data['file_date']

                # Append this DataFrame to the list
                all_dataframes.append(df)

    # Combine all DataFrames into one
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    return combined_df


In [8]:
# Path to the folder containing JSON files
folder_path = 'data'

In [9]:
# Combine all JSON files into a single DataFrame
final_dataframe = combine_json_to_dataframe(folder_path)

In [11]:
# Display the combined DataFrame
final_dataframe.head()

,id,quantity,unit_price,time_left,item.id,datetime
0,1876420303,18,63500,SHORT,190315,2023-12-22 18:19
1,1876420327,2,4400,SHORT,202070,2023-12-22 18:19
2,1876420371,9,127500,SHORT,171273,2023-12-22 18:19
3,1876420422,118,97500,SHORT,153636,2023-12-22 18:19
4,1876420519,3,700,SHORT,152631,2023-12-22 18:19
